# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846796209942                   -0.70    4.5         
  2   -7.852315815645       -2.26       -1.53    1.0   22.4ms
  3   -7.852616953688       -3.52       -2.55    1.5   25.3ms
  4   -7.852645993071       -4.54       -2.91    2.8   31.1ms
  5   -7.852646541060       -6.26       -3.25    1.0   22.5ms
  6   -7.852646679984       -6.86       -4.15    1.0   22.9ms
  7   -7.852646686643       -8.18       -5.35    2.0   28.4ms
  8   -7.852646686726      -10.08       -5.50    2.5   30.0ms
  9   -7.852646686729      -11.48       -5.90    1.0   23.3ms
 10   -7.852646686730      -12.23       -6.38    1.2   23.8ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846750531154                   -0.70           4.5         
  2   -7.852552409317       -2.24       -1.63   0.80    2.0    246ms
  3   -7.852638799638       -4.06       -2.74   0.80    1.0   21.7ms
  4   -7.852646579117       -5.11       -3.34   0.80    2.0   26.6ms
  5   -7.852646677418       -7.01       -4.27   0.80    1.0   21.1ms
  6   -7.852646686569       -8.04       -4.81   0.80    2.2   27.7ms
  7   -7.852646686719       -9.82       -5.69   0.80    1.2   22.2ms
  8   -7.852646686730      -10.99       -6.93   0.80    2.2   28.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.387562e+01     3.830957e+00
 * time: 0.05608987808227539
     1     1.435374e+00     1.867935e+00
 * time: 0.2666919231414795
     2    -1.650323e+00     2.219159e+00
 * time: 0.28975701332092285
     3    -3.977699e+00     2.037454e+00
 * time: 0.3230760097503662
     4    -5.610636e+00     1.730406e+00
 * time: 0.35573482513427734
     5    -7.116095e+00     7.771538e-01
 * time: 0.3892979621887207
     6    -7.464658e+00     6.013647e-01
 * time: 0.41211795806884766
     7    -7.670964e+00     1.860984e-01
 * time: 0.4351358413696289
     8    -7.723602e+00     1.256394e-01
 * time: 0.457927942276001
     9    -7.766271e+00     7.477208e-02
 * time: 0.4808080196380615
    10    -7.781895e+00     5.767812e-02
 * time: 0.5034949779510498
    11    -7.796630e+00     6.133094e-02
 * time: 0.526465892791748
    12    -7.811670e+00     5.428785e-02
 * time: 0.5495798587799072
    13    -7.827941e+00     4.981379e-02
 * time: 0.57264685

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846852126294                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645892920                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.86s
  3   -7.852646686730      -13.25       -7.23    157ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.2946319433704964e-7
|ρ_newton - ρ_scfv| = 9.488489954558992e-8
|ρ_newton - ρ_dm|   = 1.4311169311692976e-9
